In [ ]:
import numpy as np
import pandas as pd

np.random.seed(1993)
file_path = "./model/model_weights.hdf5"

In [ ]:
train = pd.read_json("../input/train.json")

In [ ]:
train = train[train['inc_angle'] != 'na']

In [ ]:
train_label = train.is_iceberg
train_label = np.array(train_label, dtype='float')

In [ ]:
train_data = train.drop(columns=['is_iceberg', 'id'])

In [ ]:
train_data = train_data[train_data['inc_angle'] != 'na']

In [ ]:
rows = train_data.shape[0]

In [ ]:
angle_train = np.array([x[2] for x in train_data.values])

In [ ]:
def get_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():        
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = np.sqrt(band_1**2 + band_2**2)
                
        imgs.append(np.dstack((band_1, band_2, band_3)))

    return np.array(imgs)

In [ ]:
data_bands = get_imgs(train_data)

In [ ]:
data_bands.shape

In [ ]:
data_bands_pre = data_bands.reshape(1471, 75 * 75 * 3)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler_angle = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_bands_pre.reshape(1471, 75, 75, 3)

In [ ]:
data_bands_pre = scaler.fit_transform(data_bands_pre)
angle_train = scaler_angle.fit_transform(np.matrix(angle_train).T).reshape(rows)

In [ ]:
imgs_bands = data_bands_pre.reshape((1471, 75, 75, 3))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
X, X_test, angle, angle_test, y, y_test = train_test_split(imgs_bands, angle_train, train_label, test_size=0.35, stratify=train_label)

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import metrics

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [ ]:
from keras import layers
from keras import models
from keras.constraints import maxnorm
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate

def create_model(optimizer='rmsprop', init_mode='glorot_uniform', dropout=.2, weight_constraint=4):
    bn_model = .0
    input_1 = layers.Input(shape=(75, 75, 3), name="bands")
    input_2 = layers.Input(shape=[1], name="angle")
        
    img_1 = layers.Conv2D(48, (4,4), activation='relu', input_shape=(75, 75, 3)) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = layers.MaxPooling2D((2, 2)) (img_1)
    img_1 = layers.Dropout(dropout) (img_1)
    
    img_1 = layers.Conv2D(32, (3,3), activation='relu', kernel_constraint=maxnorm(weight_constraint)) (img_1)
    img_1 = layers.MaxPooling2D((2, 2)) (img_1)
    img_1 = layers.Dropout(dropout) (img_1)
    
    img_1 = layers.Conv2D(16, (2,2), activation='relu', kernel_constraint=maxnorm(weight_constraint)) (img_1)
    img_1 = layers.MaxPooling2D((2, 2)) (img_1)
    img_1 = layers.Dropout(dropout) (img_1)
    img_1 = layers.Flatten() (img_1)    
    
    img_2 = layers.Conv2D(48, (4,4), activation='softsign') ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = layers.MaxPooling2D((2,2)) (img_2)
    img_2 = layers.Dropout(dropout) (img_2)      
    
    img_2 = layers.Conv2D(32, (3,3), activation='softsign', kernel_constraint=maxnorm(weight_constraint)) (img_2)
    img_2 = layers.MaxPooling2D((2, 2)) (img_2)
    img_2 = layers.Dropout(dropout) (img_2)
    
    img_2 = layers.Conv2D(16, (2,2), activation='softsign', kernel_constraint=maxnorm(weight_constraint)) (img_2)
    img_2 = layers.MaxPooling2D((2, 2)) (img_2)
    img_2 = layers.Dropout(dropout) (img_2) 
    img_2 = layers.Flatten() (img_2)    
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense = layers.Dropout(dropout) (layers.Dense(8, activation='relu', kernel_constraint=maxnorm(weight_constraint)) (img_concat))     
    
    output = layers.Dense(1, activation="sigmoid") (dense)
    model = models.Model([input_1,input_2],  output)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    #model.summary()
    return model

In [ ]:
callbacks = get_callbacks(filepath=file_path, patience=10)
model = create_model()

In [ ]:
model.fit([X, angle], y, 
          epochs=50, 
          validation_data=([X_test, angle_test], y_test), 
          batch_size=16, 
          callbacks=callbacks)

In [ ]:
from keras.models import model_from_json
import os

In [ ]:
model_json = model.to_json()
with open('./covnet-v7.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
json_file = open('./covnet-v7.json', 'r')
model = model_from_json(json_file.read())
json_file.close()

model.load_weights(filepath=file_path)

In [ ]:
model.save_weights('./covnet-v7.h5')

In [ ]:
test = pd.read_json('../input/test.json')

In [ ]:
print(test.shape)
rows = test.shape[0]

In [ ]:
angle_test = np.array([x[3] for x in test.values])

In [ ]:
test_bands = get_imgs(test)

In [ ]:
test_bands = test_bands.reshape(rows, 75 * 75 * 3)

In [ ]:
test_bands = scaler.transform(test_bands)
angle_test = scaler_angle.transform(np.matrix(angle_test).T).reshape(rows)

In [ ]:
test_bands = test_bands.reshape((rows, 75, 75, 3))

In [ ]:
prediction = model.predict([test_bands, angle_test], verbose=1, batch_size=16)

In [ ]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

In [ ]:
submission.to_csv('./submission.csv', encoding='utf-8', float_format='%.8f', index=False)